In [29]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3" device=0, framework="pt")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [30]:
candidate_labels = [
    "Base",
    "Fat",
    "Sweetener",
    "Binding Agent",
    "Leavening Agent",
    "Flavorings",
    "Add-ins",
    "Seasoning",
    "Texture Enhancers",
    "Decorations/Toppings",
    "Liquid",
    "Chemical Leaveners"
  ]

In [32]:
ingredients = [
    "Flour",
    "Milk",
    "Baking soda",
    "Grandfathrers ashes"
]

In [5]:
import json

In [33]:
with open("data/results/ingredients.json", encoding="utf8") as file: 

    ingredients = json.load(file)

ingrednames = set()

for ingred in ingredients:
    ingrednames.add(ingred['name'])
ingrednames


{'pasta with cream sauce',
 'orange chicken',
 'lobster bisque',
 'black bean soup',
 'ghee',
 'chicken or turkey and noodles',
 'soy chips',
 'basil',
 'spinach salad',
 'eggplant',
 'beef and noodles with soy-based sauce',
 'dukboki or tteokbokki',
 'ham and vegetables including carrots broccoli',
 'gelatin dessert',
 'kefir',
 'long island iced tea',
 'veal with vegetables including carrots',
 'tuna noodle casserole with cream or white sauce',
 'double cheeseburger mcdonalds',
 'yellow rice',
 'icing',
 'lemonade',
 'spanish rice',
 'lobster',
 'bamboo shoots',
 'kidney beans with meat',
 'stewed goat',
 'restaurant',
 'seafood soup with potatoes',
 'chard',
 'pita chips',
 'spaghetti sauce',
 'baby toddler fruit',
 'milk shake',
 'steak',
 'cereal',
 'broccoli slaw salad',
 'pine nuts',
 'banana split',
 'stuffed pork roast',
 'antipasto with ham',
 'taco or tostada salad with chicken',
 'sugar substitute',
 'alcoholic malt beverage',
 'breakfast pizza with egg',
 'shrimp and noodl

In [34]:
len(ingrednames)

1948

In [35]:
classified = classifier(
    list(ingrednames)[:10],
    candidate_labels=candidate_labels,
)

In [37]:
full_ingredients = {}

cutoffPoint = 0.125

for classi in classified:
    assignedClasses = []
    for i in range(len(classi["labels"])):
        label = classi["labels"][i]
        probability = classi["scores"][i]
        if(i == 0 or probability > cutoffPoint):
            assignedClasses.append(label)
    full_ingredients[classi["sequence"]] = {"cookiecat" : assignedClasses}

full_ingredients

{'pasta with cream sauce': {'cookiecat': ['Base',
   'Texture Enhancers',
   'Add-ins']},
 'orange chicken': {'cookiecat': ['Add-ins']},
 'lobster bisque': {'cookiecat': ['Liquid', 'Fat']},
 'black bean soup': {'cookiecat': ['Texture Enhancers',
   'Base',
   'Add-ins',
   'Liquid']},
 'ghee': {'cookiecat': ['Sweetener']},
 'chicken or turkey and noodles': {'cookiecat': ['Add-ins',
   'Texture Enhancers',
   'Base']},
 'soy chips': {'cookiecat': ['Base', 'Binding Agent', 'Decorations/Toppings']},
 'basil': {'cookiecat': ['Base']},
 'spinach salad': {'cookiecat': ['Texture Enhancers', 'Base', 'Add-ins']},
 'eggplant': {'cookiecat': ['Flavorings', 'Texture Enhancers', 'Base']}}

In [38]:
for i in range(len(ingredients)):
    ingred = ingredients[i]
    if ingred["name"] in full_ingredients:
        ingredients[i]["bakingrole"] = full_ingredients[ingred['name']]['cookiecat']
    else:
        print(ingred["name"] + " not found?")

ingredients

hummus not found?
tomatoes not found?
beans not found?
frankfurter not found?
nuts not found?
kale not found?
egg not found?
egg not found?
egg not found?
onion rings not found?
pickles not found?
cheese not found?
cheese not found?
grapefruit juice not found?
peaches not found?
seeds not found?
bread not found?
kale not found?
mustard not found?
kiwifruit not found?
nectarines not found?
cheese not found?
cheese not found?
cheese not found?
egg not found?
egg not found?
egg not found?
yogurt not found?
yogurt not found?
oil not found?
chicken not found?
chicken not found?
sauce not found?
ham not found?
olives not found?
cookies not found?
tomatoes not found?
fish not found?
fish not found?
fish not found?
restaurant not found?
restaurant not found?
restaurant not found?
bread not found?
beef not found?
beef not found?
beef not found?
beef not found?
beef not found?
beef not found?
carrots not found?
cheese not found?
cheese not found?
cheese not found?
figs not found?
lettuce not fou

[{'name': 'hummus',
  'tags': ['commercial'],
  'nutrients': [{'name': 'Cryptoxanthin, beta', 'amount': 3, 'unit': 'µg'},
   {'name': 'Tocopherol, delta', 'amount': 1.3, 'unit': 'mg'},
   {'name': 'Thiamin', 'amount': 0.15, 'unit': 'mg'},
   {'name': 'Riboflavin', 'amount': 0.115, 'unit': 'mg'},
   {'name': 'Folate, total', 'amount': 36, 'unit': 'µg'},
   {'name': 'Vitamin K (phylloquinone)', 'amount': 17.2, 'unit': 'µg'},
   {'name': 'Fatty acids, total trans', 'amount': 0.018, 'unit': 'g'},
   {'name': 'Fatty acids, total saturated', 'amount': 2.22, 'unit': 'g'},
   {'name': 'SFA 14:0', 'amount': 0.009, 'unit': 'g'},
   {'name': 'SFA 22:0', 'amount': 0.044, 'unit': 'g'},
   {'name': 'SFA 17:0', 'amount': 0.01, 'unit': 'g'},
   {'name': 'SFA 24:0', 'amount': 0.027, 'unit': 'g'},
   {'name': 'MUFA 24:1 c', 'amount': 0.005, 'unit': 'g'},
   {'name': 'MUFA 18:1 c', 'amount': 6.25, 'unit': 'g'},
   {'name': 'PUFA 18:2 n-6 c,c', 'amount': 6.81, 'unit': 'g'},
   {'name': 'MUFA 22:1 c', 'amo

In [43]:
with open("data/results/ingredients_roles.json","w", encoding="utf8") as file: 

    ingredients = json.dump(ingredients, file)